In [2]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [3]:
import sys
sys.path.append('../../../')

from tensorflow_models import CNNClassifier
import tensorflow as tf

In [4]:
processedDataX_path = "../../preprocessedSamples_cut_500_X_samples_allGuitar_20480_Mm7_R1D.data"
processedDatay_path = "../../preprocessedSamples_cut_500_y_samples_allGuitar_20480_Mm7_R1D.data"
processedData_path = ""

if os.path.isfile(processedDataX_path): #if already preprocessed
    processedX = np.load(processedDataX_path)
    processedy = np.load(processedDatay_path)
else:
    import pandas as pd
    import librosa
    df = pd.read_csv('../../samples_allGuitar_20480_Mm7_R1D.csv')

    X_load = np.array(df.iloc[:,:-1], dtype=np.float)
    y_load = np.array(df.iloc[:,-1], dtype=np.float)
    processedX = np.zeros((len(X_load),12,80,1), dtype=np.float)
    processedy = np.zeros(len(y_load), dtype=np.float)
    X_fft_new = np.zeros(20480)
    for i in range(len(X_load)):
        sample = np.fft.rfft(X_load[i])
        for ii in range(len(sample)):
            if ii < 500: #ignore frequencies greater than 2kHz
                X_fft_new[ii] = sample[ii]
                
        sample = np.fft.ifft(X_fft_new)
        sample = librosa.feature.chroma_stft(y=sample, sr=44100, n_fft=20480, hop_length=258)
        sample = np.atleast_3d(sample)
        processedX[i] = sample
        processedy[i] = y_load[i]
        if i % 400 == 0:
            print(i)

    processedX.dump(processedDataX_path)
    processedy.dump(processedDatay_path)

from sklearn.utils import shuffle
print(processedy)
sprocessedX, sprocessedy = shuffle(processedX, processedy)
print(len(sprocessedX))

trainRange = int(len(sprocessedX) * 0.8)
validRange = int(len(sprocessedX) * 0.9)
testRange = int(len(sprocessedX) * 0.1)


X_train = np.array(sprocessedX[:trainRange], dtype=np.float)
y_train = np.array(sprocessedy[:trainRange], dtype=np.float)

X_valid = np.array(sprocessedX[trainRange:validRange], dtype=np.float)
y_valid = np.array(sprocessedy[trainRange:validRange], dtype=np.float)

X_test = np.array(sprocessedX[testRange:], dtype=np.float)
y_test = np.array(sprocessedy[testRange:], dtype=np.float)
print(y_test[1])
print(sprocessedX.shape)

[  0.   0.   0. ...,  45.  45.  45.]
3214
18.0
(3214, 12, 80, 1)


In [7]:
def leaky_relu(alpha=0.01):
	def parametrized_leaky_relu(z, name=None):
		return tf.maximum(alpha * z, z, name=name)
	return parametrized_leaky_relu

model = CNNClassifier(n_hidden_layers=1, n_neurons=400, optimizer_class=tf.train.AdamOptimizer,
                    learning_rate=0.05, batch_size=200, activation=leaky_relu(), dropout_rate=0.1,
                    conv1={'conv1_fmaps': 16, 'conv1_ksize': 5, 'conv1_stride': 1, 'conv1_dropout': 0.3, 'conv1_activation': tf.nn.relu,
                    conv2={'conv2_fmaps': 16, 'conv2_ksize': 5, 'conv2_stride': 1, 'conv2_dropout': 0.3, 'conv2_activation': tf.nn.relu,
                    architecture=1)

model.fit(X_train, y_train, X_valid=X_valid, y_valid=y_valid)
model.accuracy_score(X_test, y_test)

trained_model_file = '../../../saved_models/saved-model-final.ckpt'


model.save(trained_model_file)

SyntaxError: invalid syntax (<ipython-input-7-5d7a55e34d5e>, line 9)